In [ ]:
# Necessary Libraries for RAG Chain
from langchain.chains.combine_documents import create_stuff_documents_chain     # Combines text chunks into a query, stuffs them together into a single prompt template
from langchain.chains import create_retrieval_chain                             # Creates a Retrieval-Augmented Generation (RAG) chain with a retriever and document chain
from langchain_groq import ChatGroq                                             # Use to run open-source models at very high speed
from langchain import hub                                                       # Hub to pull prompt templates

# Necessary Libraries for Vector Database
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

from dotenv import load_dotenv
import os

/Users/jkhang/Documents/GitHub/Artificial-Intelligence-Medical-Chatbot/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jkhang/Documents/GitHub/Artificial-Intelligence-Medical-Chatbot/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load API Key from .env file
load_dotenv()

# Constant Variables
GROQ_MODEL_NAME = "llama-3.1-8b-instant"
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

VECTOR_DATABASE_PATH = "./vector_database/db_faiss"

In [3]:
# Setup Large Language Model
model = ChatGroq(model = GROQ_MODEL_NAME, temperature = 0.5, max_tokens = 512, api_key = GROQ_API_KEY)

In [4]:
# Create Vector Embeddings
embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

# Load Dataset from FAISS Vector Database
database = FAISS.load_local(VECTOR_DATABASE_PATH, embedding_model, allow_dangerous_deserialization = True)

In [ ]:
# Pull Pre-Build Prompt Template from LangChain Hub
prompt = hub.pull("langchain-ai/retrieval-qa-chat")

# Combine Text Chunks with Prompt Template
docs_chain_combine = create_stuff_documents_chain(llm = model, prompt = prompt)

# Build a Retrieval-Augmented Generation (RAG) Chain
rag_chain = create_retrieval_chain(database.as_retriever(search_kwargs = {"k": 3}), docs_chain_combine)


In [6]:
# Invoke the QA Chain with a sample question
que = "What are the common symptoms of diabetes?"
res = rag_chain.invoke({"input": que})

print("ANSWER: ", res["answer"])
print("\nSOURCE DOCUMENTS:")
for doc in res["context"]:
    print(f"- {doc.metadata} -> {doc.page_content[:200]}...")

ANSWER:  The common symptoms of diabetes include:

1. Feeling tired and sick
2. Frequent urination
3. Excessive thirst
4. Excessive hunger
5. Weight loss
6. Lethargy
7. Slow wound healing
8. Urinary tract infections
9. Gum disease
10. Blurred vision

SOURCE DOCUMENTS:
- {'producer': 'GPL Ghostscript 9.10', 'creator': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'moddate': '2017-05-01T10:37:35-07:00', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'source': 'data/The-Glae-Encyclopedia-of-Medicine.pdf', 'total_pages': 759, 'page': 436, 'page_label': '437'} -> diabetes symptoms include isoniazid, nicotinic acid,
cimetidine, and heparin.
Symptoms
Symptoms of diabetes can develop suddenly (over
days or weeks) in previously healthy children or adoles-
cents, o...
- {'producer': 'GPL Ghostscript 9.10', 'creator': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'moddate': '2017-05-01T10:37:35-07:00', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'source': 'data/The-